In [ ]:
import fitsio
import proplot as pplt
import numpy as np

In [ ]:
import subprocess

subprocess.run(
    '''\
rm -f qadata*.fits && easyaccess --db desoper \
-c "select * from (\
 select qa.*, dbms_random.value as rnd from PIFF_HSM_MODEL_QA qa\
) r where r.rnd < 1.1; \
> qadata.fits"\
 ''',
    shell=True,
    check=True,
)

In [ ]:
import glob

fnames = glob.glob("qadata*.fits")
dall = [fitsio.read(f, lower=True) for f in fnames]
dall = np.hstack(dall)
dall.dtype.names

In [ ]:
d = dall[(dall["flag"] == 0) & (dall["nstar"] >= 30)]

In [ ]:
fig, axs = pplt.subplots(nrows=1, ncols=3, share=0)

ax = axs[0]
ax.hist(
    np.abs(d["star_t_mean"] - d["exp_star_t_mean"])/d["exp_star_t_std"],
    bins=50,
    log=True
);
ax.set_xlabel("|T-Mean(Texp)|/Std(Texp)")
ax.axvline(4, color="k")
frac = np.mean(np.abs(d["star_t_mean"] - d["exp_star_t_mean"])/d["exp_star_t_std"] > 4)
print("exposure cut:", frac)
ax.set_title("cut: %0.3f%%" % (frac*100))

ax = axs[1]
ax.hist(
    d["star_t_std"]/d["star_t_mean"],
    bins=50,
    log=True
);
frac = np.mean(d["star_t_std"]/d["star_t_mean"] >= 0.3)
print("star T cut:", frac)
ax.set_xlabel("Std(T)/Mean(T)")
ax.axvline(0.3, color="k")
ax.set_title("cut: %0.3f%%" % (frac*100))

ax = axs[2]
ax.hist(
    d["fwhm_cen"],
    bins=50,
    log=True
);
frac = np.mean(d["fwhm_cen"] > 3.6)
print("fwhm_cen cut:", frac)
ax.set_xlabel("fwhm_cen")
ax.axvline(3.6, color="k")
ax.set_title("cut: %0.3f%%" % (frac*100))
